# Моделирование почтового отделения

На примере простой задачи моделирования работы почтового (к примеру) отделения с несколькими окнами обслуживания клиентов разберёмся в работе с разделяемыми ресурсами в SimPy.
Саму задачу сформулируем следующим образом.

Имеется почтового отделение с 3 окнами.
Каждое окно имеет свой график работы: массив вида `[(ts_1, te_1), (ts_2, te_2), ...]`, где `ts_i` и `te_i` — глобальное время начала и конца работы окна в `i`-ой сессии соответственно.

Клиенты приходят в случайные моменты времени, причём время между двумя клиентами распределено по экспоненциальному закону

$$
\Delta t_\text{a} \sim \frac{1}{\tau_\text{a}} \exp\left(-\frac{1}{\tau_\text{a}} \right)
$$

со средним временем (`mean_arrival`) между клиентами $\tau_\text{a}$.
Время обслуживания (`duration`) каждого клиента случайно и распределено равномерно от 1 минуты до 10 минут (модельные часы будут отсчитывать время в часах).

В качестве результата выведем все события, которые происходят в системе.

## Программная реализация

Полный код представлен ниже.
Изучите его, обдумайте, проанализируйте, попытайтесь понять, прежде чем переходить к последующим пояснениям.

In [2]:
import random as rand
import simpy as sim
from collections import namedtuple


# Простая структура данных клиента
Client = namedtuple("Client", "cid arrival duration")


def clients_arriving(env: sim.Environment,
                     clients: sim.Store,
                     mean_arrival: float):
    """Процесс прихода клиентов.

    * env: экземпляр среды SimPy
    * clients: разделяемый ресурс - очередь клиентов
    * mean_arrival: среднее время между приходами клиентов
    """
    # cid - индивидуальный номер клиента
    cid = 1

    while True:
        # Ожидаем нового клиента через случайное время
        yield env.timeout(rand.expovariate(1/mean_arrival))
        # При наступлении события выполняем callback-код:
        # создаём нового клиента
        new_client = Client(cid, env.now, rand.uniform(1/60, 1/6))
        print(f"{env.now:.4f}: Клиент #{new_client.cid} ПРИШЁЛ")

        # Ожидаем возможности добавить клиента в очередь:
        # в данном случае выполняется сразу,
        # поскольку размер очереди не ограничен
        yield clients.put(new_client)
        # После выполняется callback-код
        cid += 1


def window(env: sim.Environment,
           timed_agenda: list[tuple[float, float]],
           clients: sim.Store,
           win_num: int):
    """Процесс работы окошка.

    * env: экземпляр среды SimPy
    * timed_agenda: расписание работы окна
    * clients: очередь клиентов (разделяемый ресурс)
    * win_num: номер окошка
    """
    # ts, te - время открытия и закрытия окошка
    for ts, te in timed_agenda:
        # Ожидание открытия окна
        yield env.timeout(ts - env.now)
        print(
            f"{env.now:.4f}: Окно #{win_num} ОТКРЫЛОСЬ"
        )

        # Ожидание окончания процесса обслуживания
        # клиентов (очереди clients).
        # Процесс завершится, когда окошко закроется на перерыв
        yield env.process(service(env, clients, te, win_num))
        # Внутри service бесконечный цикл, который прерывается,
        # когда окошко закрывается на перерыв.
        # Это приводит к новой итерации цикла for =>
        # позже окошко откроется по расписанию в момент времени ts


def service(env: sim.Environment,
            clients: sim.Store,
            when_close: float,
            win_num: int):
    """Процесс обслуживания клиентов до тех пор,
    пока окошко не закроется на перерыв в момент времени
    `when_close` или после обслуживания последнего клиента,
    вышедшего за рамки данного времени.

    * env: экземпляр среды SimPy
    * clients: очередь клиентов (разделяемый ресурс)
    * when_close: когда окошку закрываться на перерыв
    * win_num: номер обслуживающего окошка
    """
    while True:
        # Запрос на получение ресурса
        with clients.get() as client_req:
            dt = when_close - env.now
            if dt < 0:
                # Условие закрытия окна
                print(f"{env.now:.4f}: Окно #{win_num} ЗАКРЫЛОСЬ")
                return
            close = env.timeout(dt)
            # Ожидаем либо получения ресурса (клиента),
            # либо наступления перерыва
            event = yield client_req | close
            # В event хранится словарь {событие: значение_события}

            # Если получили доступ к ресурсу:
            if client_req in event:
                # - достаём соответствующий экземпляр клиента
                client: Client = client_req.value
                print(
                    f"{env.now:.4f}: Окно #{win_num} ЗАНЯТО "
                    f"Клиентом #{client.cid}"
                )

                # - ожидаем, пока клиент обслуживается
                yield env.timeout(client.duration)
                print(f"{env.now:.4f}: Окно #{win_num} СВОБОДНО")
            else:
                # - ещё одно условие закрытия окна
                print(f"{env.now:.4f}: Окно #{win_num} ЗАКРЫЛОСЬ")
                return


# Затравка для воспроизведения случайных чисел
rand.seed(42)
# Инициализируем среду SimPy с указанием начального времени
env = sim.Environment(initial_time=8.5)
# Назначаем расписание работы трёх окон
windows_work_time = [
    [(8.5, 10.5), (11.5, 13.5)],
    [(8.5, 9.5), (10.5, 12.5)],
    [(11.5, 14.5)]
]

# Инициализируем разделяемый ресурс - очередь клиентов
clients = sim.Store(env)
# Запускаем процесс прихода клиентов
env.process(
    clients_arriving(env, clients, mean_arrival=1/3)
)
# Запускаем три процесса работы окон.
# Каждое окно со своим расписанием работы timed_agenda
for i, timed_agenda in enumerate(windows_work_time):
    # Обратите внимание, что одну и ту же функцию, но с разными
    # аргументами, мы используем, чтобы создать
    # три различных процесса (для каждого окна)
    env.process(window(env, timed_agenda, clients, i+1))

# Запускаем симуляцию
env.run(until=13.5)

8.5000: Окно #1 ОТКРЫЛОСЬ
8.5000: Окно #2 ОТКРЫЛОСЬ
8.8400: Клиент #1 ПРИШЁЛ
8.8400: Окно #1 ЗАНЯТО Клиентом #1
8.8604: Окно #1 СВОБОДНО
8.9472: Клиент #2 ПРИШЁЛ
8.9472: Окно #2 ЗАНЯТО Клиентом #2
8.9974: Окно #2 СВОБОДНО
9.3918: Клиент #3 ПРИШЁЛ
9.3918: Окно #1 ЗАНЯТО Клиентом #3
9.5000: Окно #2 ЗАКРЫЛОСЬ
9.5099: Окно #1 СВОБОДНО
10.1342: Клиент #4 ПРИШЁЛ
10.1342: Окно #1 ЗАНЯТО Клиентом #4
10.1639: Окно #1 СВОБОДНО
10.3169: Клиент #5 ПРИШЁЛ
10.3169: Окно #1 ЗАНЯТО Клиентом #5
10.3380: Окно #1 СВОБОДНО
10.3991: Клиент #6 ПРИШЁЛ
10.3991: Окно #1 ЗАНЯТО Клиентом #6
10.4081: Клиент #7 ПРИШЁЛ
10.4916: Окно #1 СВОБОДНО
10.4916: Окно #1 ЗАНЯТО Клиентом #7
10.5000: Окно #2 ОТКРЫЛОСЬ
10.5381: Окно #1 СВОБОДНО
10.5381: Окно #1 ЗАКРЫЛОСЬ
10.7579: Клиент #8 ПРИШЁЛ
10.7579: Окно #2 ЗАНЯТО Клиентом #8
10.8409: Клиент #9 ПРИШЁЛ
10.8563: Окно #2 СВОБОДНО
10.8563: Окно #2 ЗАНЯТО Клиентом #9
10.9614: Окно #2 СВОБОДНО
11.3935: Клиент #10 ПРИШЁЛ
11.3935: Окно #2 ЗАНЯТО Клиентом #10
11.4111: Окно #2 СВОБ

Итак, большая часть кода вам уже, наверняка, понятна.
Рассмотрим основную особенность — наличие разделяемого ресурса `clients = sim.Store(env)`.

Всего в SimPy есть три вида ресурсов:

1. [`Resource`](https://simpy.readthedocs.io/en/latest/topical_guides/resources.html#res-type-resource) — по сути своей это классический [семафор](https://ru.wikipedia.org/wiki/Семафор_(программирование)). Ресурсы могут использоваться ограниченным числом процессов одновременно (например, заправочная станция с ограниченным числом топливных насосов). Процессы запрашивают (`Resource.request()`) эти ресурсы, чтобы стать пользователем ("владеть" ими) и должны освободить их (`Resource.release()`), как только они закончатся (например, транспортные средства прибывают на заправочную станцию, используют топливный насос, если он доступен, и уезжают, как заправятся).
2. [`Container`](https://simpy.readthedocs.io/en/latest/topical_guides/resources.html#res-type-container) — хранят в себе количество чего-либо однородного (объём бензина в цистерне заправки, например, или количество грибов в корзине).
3. [`Store`](https://simpy.readthedocs.io/en/latest/topical_guides/resources.html#stores) — "магазин" близок к контейнеру, но может хранить разнородные объекты, как настоящий магазин, в котором можно купить и фрукты, и овощи и прочие продукты.\

У `Store` и `Container` нет методов `request()` и `release()`, как у `Resource`.
Вместо них методы добавления в очередь `put(item)` и получения следующего объекта в очереди `get()`.

В нашей задаче разделяемым ресурсом является очередь клиентов `clients`.
Этот ресурс используется в двух процессах: `clients_arriving` и `service`.
В `clients_arriving` в очередь добавляется (`yield clients.put(client)`) новый клиент, а в `service` — окно запрашивает ресурс через менеджер контекста `with`: `with clients.get() as client_req`.

```{note}
Использование менеджера контекста при запросе любого ресурса в SimPy является хорошим и надёжным подходом, поскольку после захвата ресурса **крайне важно ресурс освобождать**.
Менеджер контекста берёт эту работу на себя.
Вручную же делать это неудобно по ряду причин, которые станут вам понятны в процессе накопления опыта моделирования с использованием SimPy.
Коротко говоря, возможно такое, что у вас наступит другое событие вместо события получения ресурса.
Если вы явно не освободите ресурс, то он будет потерян для всех остальных процессов.
```

Важным нововведением является использование *условного события* `event = yield client_req | close`.
По условию задачи каждое окно работает по своему расписанию.
Следовательно, каждое свободное работающее окно ждёт одно событие из двух: либо до закрытия окна придёт клиент, либо наступит время перерыва.
Именно это и означает запись `event = yield client_req | close` (обратите внимание на наличие `yield`).
В первом случае окно получает клиента и должно его обслужить в течение времени `duration` этого клиента.
Во втором случае окно закрывается на перерыв.

Условие `if client_req in event` записывается именно так потому, что `event` представляет собой обычный Python-словарь.
Ключами являются наступившие события, а значениями — значения (`value`) этих событий.
Если среди ключей есть `client_req` (то есть в очереди есть хотя бы один клиент), то он забирается из очереди и обслуживается.
В ином случае наступило событие `close` — окно закрывается на перерыв.
Это выражается в завершении генераторной функции `service` оператором `return`.

Результат работы программы вы можете видеть под исходным кодом.
Убедитесь в правильности вывода.
Подобную модель можно использовать для решения довольно интересных оптимизационных задач.